# Midterm 2

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from arch import arch_model
from arch.univariate import GARCH, EWMAVariance 
from sklearn import linear_model
import scipy.stats as stats
from statsmodels.regression.rolling import RollingOLS
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.precision", 4)
sns.set(rc={'figure.figsize':(15, 10)})

In [2]:
file_path = 'C:/Users/dcste/OneDrive/Portfolio_Theory/Homework_Jupyter/portfolio_theory/midterm_2.xlsx'
assets = pd.read_excel(file_path, sheet_name = 'assets (excess returns)').set_index("Date")
factors = pd.read_excel(file_path, sheet_name = 'factors (excess returns)').set_index("Date")
factors.head()

,MKT,RMW
Date,,
2009-04-30,0.1018,0.0131
2009-05-31,0.0521,-0.0078
2009-06-30,0.0043,-0.0141
2009-07-31,0.0772,-0.0046
2009-08-31,0.0333,-0.0302


# Part 1

# 1.1

The Value Factor is long stocks with high book-to-market ratios and short stocks with low book-to-market ratios (growth stocks). This makes net-investment zero and captures the returns that investors would benefit from investing in *value stocks.* This construction reduces the correlation between the market and size factor because by definition, the value portfolio is trying to harvest the premium on value stocks and nothing else.


# 1.2
This would make Value a great risk diversifier in any portfolio. You could defend the value factor by saying its very uncorrelated. 

# 1.3

- AQR would conclude that the size factor does not provide any diversification benefits because of its correlation to the market factor and low expected excess return thus AQR might conclude to eliminate SMB as a factor altogether. 

# 1.4

- If the CAPM model worked perfectly, then True the Market Factor would have the highest sharpe ratio. This means alpha must be 0 and the r-squared is 1. The market portfolio will be on the efficient frontier and tangent point to the capital asset pricing line.

# 1.5

- The biggest difference in the performance of the long-only momentum and long-short momentum is that the long-only portfolio generates very high returns, however, unfortunately it is very correlated to the market making it not a great diversifier. 

# 1.6

- No we would not be able to operate an effective momentum strategy. The R-squared is very small and beta indicates a slight autocorrelation. In order to create an effective strategy we need to make thousands of trades, but operating at a high volume of trades is unrealistic because of trading costs. So this would be a terrible strategy. 

# 1.7

- The UMD factor was determined that it can have a positive net exposure to the market whereas SMB and HML must have 0 next exposure to the market. The reason why UMD can is because while originally it was 0 net exposure, AQR changed the UMD index to make it more realistic. They developed three separate long-only momentum indices that held stocks with high liquidity and was not rebalanced monthly. This will make the new UMD indec have a positive exposure. 


# 1.8

- In order for a Linear Factor Pricing Model to be considered perfect, the betas of the factors need to explain 100% of the variance in the asset it is trying to price. This means the r-squared needs to be one. Additionally, the alpha must be zero, meaning there are no other factors that can price the asset, otherwise arbitrage would exist. 
  
# 1.9

No it is impossible. 




# Section 2

- Estimating the time-series test of the pricing model. 

# 2.a

In [3]:
ts_df = pd.DataFrame(data = None, index = assets.columns, columns = ['Alpha', 'MKT','RMW','R-squared'])

for asset in assets.columns:
    X = sm.add_constant(factors)
    y = assets[asset]
    ts_reg = sm.OLS(y,X).fit()
    ts_df.loc[asset] = [ts_reg.params[0]*12, ts_reg.params[1]*12,ts_reg.params[2]*12, ts_reg.rsquared]
    
 
ts_df



,Alpha,MKT,RMW,R-squared
BWX,-0.0406,2.8355,1.2284,0.2047
DBC,-0.052,7.1182,-1.3031,0.2568
EEM,-0.0689,11.3234,-1.1377,0.5601
EFA,-0.0582,11.1751,-0.9298,0.7622
HYG,-0.0004,5.4388,0.4338,0.5886
IEF,0.024,-0.9819,2.032,0.082
IYR,0.001,10.9603,3.2776,0.5514
PSP,-0.0659,16.0891,-1.3174,0.8393
QAI,-0.0237,3.339,0.0341,0.7174
SPY,0.0006,11.5544,1.5102,0.9924


- Reporting the annualized MAE and mean r-squared statistics

In [4]:
print('The annuaized mean MAE is ', round(ts_df['Alpha'].abs().mean(),4))
print("The average R-squared is", round(ts_df['R-squared'].mean(), 4))

The annuaized mean MAE is  0.0311
The average R-squared is 0.5155


# 2.b

- If the pring model worked perfectly we would expect the MAE to be extremly close to zero or be zero in order to suggest evidence that these factors accurately price the assets. This would be sufficient to explain that the factors can explain most of the variation in the portfolio returns. 

# 2.c

Overall, I would say that the time-series test does not do a great job at explaining the portfolio returns. On average, the factors cannot explain 300 basis points of portfolio returns a year and from visual inspection the market factor receives a large beta for each portfolio whereas RMW's beta is small. This would lead me to conclude that this is a poor linear factor decomposition. 

# 2.a

- Estimating the cross sectional test

In [5]:
X_cs = sm.add_constant(ts_df[['MKT','RMW']]).astype('float')
dependent = assets.mean()
cs_reg = sm.OLS(dependent,X_cs).fit()
cs_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.906
Model:                            OLS   Adj. R-squared:                  0.882
Method:                 Least Squares   F-statistic:                     38.34
Date:                Mon, 07 Nov 2022   Prob (F-statistic):           7.96e-05
Time:                        20:00:23   Log-Likelihood:                 58.108
No. Observations:                  11   AIC:                            -110.2
Df Residuals:                       8   BIC:                            -109.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0018      0.001     -2.005      0.080      -0.004       0.000
MKT            0.0008   9.46e-05      8.707      0.000       0.001       0.001
RMW            0.0015      0.000      4.621      0.002       0.001       0.002
==============================================================================
Omnibus:                        1.955   Durbin-Watson:                   1.471
Prob(Omnibus):                  0.376   Jarque-Bera (JB):                0.507
Skew:                          -0.515   Prob(JB):                        0.776
Kurtosis:                       3.214   Cond. No.                         18.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [6]:
cs_reg_df = pd.DataFrame(data = None, index = ['Alpha','MKT_beta','RMW_beta','Rsquared',"MAE"], columns = ['Cross Sectional Test'])

cs_reg_df.loc['Alpha'] = cs_reg.params[0]*12
cs_reg_df.loc['MKT_beta'] = cs_reg.params[1]*12
cs_reg_df.loc['RMW_beta'] = cs_reg.params[2]*12
cs_reg_df.loc['Rsquared'] = cs_reg.rsquared
cs_reg_df.loc['MAE'] = cs_reg.resid.abs().mean()*12

cs_reg_df

,Cross Sectional Test
Alpha,-0.0214
MKT_beta,0.0099
RMW_beta,0.0175
Rsquared,0.9055
MAE,0.0109


# 2.b

If the pricing model worked perfectly alpha would equal 0 and the r-squared would equal 1. This would also mean the MAE would also equal 0. 

# 2.c

- The factor model prices these assets very well. The R-squared is 90% meaning 90% of the variation of the assets returns are explained by the two factors. Additionally, alpha is statistically insignifcant from 0 as given by its t-distribution. Overall, we believe these model is robust and provides an investor with accurate beta risk dynamics across all the assets. 

# 2.d


In [7]:
# Expected Excess Returns

expected_return = cs_reg.params[0]+(ts_df[['MKT','RMW']]*cs_reg.params[1:]).sum(axis =1)
expected_return = expected_return *12
expected_return

BWX    0.0281
DBC    0.0261
EEM    0.0706
EFA    0.0728
HYG    0.0399
IEF    0.0044
IYR    0.1443
PSP    0.1146
QAI    0.0122
SPY    0.1193
TIP    0.0265
dtype: float64

# 3

- Yes, according to the model the RMW factor is more lucrative right now given that it has a higher risk premium in the market. Factor pricing equilibrium would say that for each additional unit of RMW beta invested, the expected yearly return to your portfolio would increase by 1.75%.

# Section 3

- Calculating the tangency portfolio of the 11 returns based on excess returns

In [8]:
def tangency_portfolio_rfr(asset_return,cov_matrix):
    """ 
        Returns the tangency portfolio weights in a (1 x n) vector when a riskless assset is available
        Inputs: 
            asset_return - Excess return over the risk free rate for each asset (n x 1) Vector
            cov_matrix = nxn covariance matrix for the assets
    """
    asset_cov = np.array(cov_matrix)
    inverted_cov= np.linalg.inv(asset_cov)
    one_vector = np.ones(len(cov_matrix.index))
    
    den = (one_vector @ inverted_cov) @ (asset_return)
    num =  inverted_cov @ asset_return
    return (1/den) * num
def portfolio_stats(df_tilde, omega, annualize_fac):
    # Mean
    mean = df_tilde.mean() @ omega * annualize_fac

    # Volatility
    vol = (df_tilde @ omega).std() * np.sqrt(annualize_fac)


    # Sharpe ratio
    sharpe_ratio = mean / vol

    return round(pd.DataFrame(data = [mean, vol, sharpe_ratio], 
                              index = ['Mean', 'Volatility', 'Sharpe'], 
                              columns = ['Portfolio Stats']), 4)

In [9]:
tan_weights = tangency_portfolio_rfr(asset_return = assets.mean(), cov_matrix = assets.cov())

tang_weights = pd.DataFrame(tan_weights, index = assets.columns, columns = ['Tangency Weights'])
tang_weights

,Tangency Weights
BWX,-16.6834
DBC,2.6617
EEM,5.0023
EFA,-1.6528
HYG,15.8455
IEF,31.6714
IYR,-4.2510
PSP,-6.9802
QAI,-60.2653
SPY,33.1711


In [10]:
portfolio_stats(assets,tan_weights,12)

,Portfolio Stats
Mean,4.2191
Volatility,2.2914
Sharpe,1.8413


# 3.2


In [11]:
tan_cs = tangency_portfolio_rfr(expected_return,assets.cov())
tang_cs = pd.DataFrame(tan_cs, index = expected_return.index, columns = ['Tangency CS'])
tang_cs

,Tangency CS
BWX,-0.6979
DBC,0.0382
EEM,-0.1377
EFA,0.3077
HYG,0.0288
IEF,0.4635
IYR,-0.1254
PSP,0.3733
QAI,4.0981
SPY,-1.9412


In [12]:
portfolio_stats(assets,tan_cs,12)

,Portfolio Stats
Mean,-0.1859
Volatility,0.1355
Sharpe,-1.3720
